# RQ4: Minimal Detectable Slowdowns

In [1]:
%load_ext autoreload
%autoreload 2

import plotly.express as px
import pandas as pd
import numpy as np
from IPython.display import display

from duet.process import *
from duet.constants import *
from common import *

df_prep = preprocess_data(load_raw())
unique_suites = df_prep[RF.suite].unique()

/Users/tdrozdik/school/asynchronous-duet/duet/process.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[max_iteration] = df.groupby(ARTIFACT_COL + RUN_ID_COL)[RF.iteration].transform(


## Arbitrary slowdown verification

In [2]:
df = df_prep[
    (
        (df_prep[RF.suite].isin(["dacapo"]))
        & (df_prep[RF.benchmark].isin(["avrora"]))
        & (df_prep[RF.runid] == 0)
        & (df_prep[RF.type].isin(["duet"]))
        & (df_prep[AF.date] == df_prep[AF.date].iloc[0])
        # & (df_prep[RF.pair] == "B")
    )
]
df = df[df[RF.iteration] < df[RF.iteration].min() + 5]

# display(convert_ns(df)[ARTIFACT_COL + ITER_ID_COL + TIME_D_COL])
display(
    px.timeline(
        df,
        x_start=RF.start,
        x_end=RF.end,
        y=RF.pair,
        color=RF.iteration,
        hover_data=[RF.benchmark, RF.pair, RF.runid, RF.time],
        title="Original",
    )
)

slowdown = 0.2
df = alter_score(df, slowdown)
df = convert_ns(df)
# display(df[ARTIFACT_COL + ITER_ID_COL + TIME_D_COL])

px.timeline(
    df,
    x_start=RF.start,
    x_end=RF.end,
    y=RF.pair,
    color=RF.iteration,
    hover_data=[RF.benchmark, RF.pair, RF.runid, RF.time],
    title=f"Slowdown of B by {(slowdown * 100):.0f}%",
)

### Computed CI on 10% slowdown

In [3]:
slowdown = 0.1
df = alter_score(df_prep, slowdown)
df = compute_ci(df)

for suite in unique_suites:
    suite_mask = df[RF.suite] == suite
    if suite_mask.any():
        fig = px.scatter(
            df[suite_mask],
            x="benchmark",
            y="mid",
            error_y="err",
            color=RF.type,
            facet_col=DF.env,
            title=f"CI with A/B emulated {(slowdown * 100):.0f}% slowdown - {suite}",
        )
        fig.update_xaxes(categoryorder="category ascending")
        fig.show()

## Slowdown computation

In [4]:
slowdowns = [0.01 * i for i in range(16)]
dfs = []
for slowdown in slowdowns:
    df = alter_score(df_prep, slowdown)
    df_ci = compute_ci(df)
    df_pred = arbiter(df, df_ci)
    df_pred = group_predictions(df_pred)
    df_pred[DF.slowdown] = slowdown
    dfs.append(df_pred)

df_slowdown = pd.concat(dfs)
df_slowdown

,environment,suite,type,total_count_ci,match_ci,err_ci,total_count_utest,match_utest,match_ci_%,match_utest_%,slowdown
0,AWS:t3.medium,dacapo,duet,10,10,0.000000e+00,10,7,1.000000,0.700000,0.00
1,AWS:t3.medium,dacapo,seqn,10,9,-8.817293e+07,10,5,0.900000,0.500000,0.00
2,AWS:t3.medium,dacapo,syncduet,10,9,3.281199e+06,10,5,0.900000,0.500000,0.00
3,AWS:t3.medium,renaissance,duet,24,23,-3.718924e+08,24,20,0.958333,0.833333,0.00
4,AWS:t3.medium,renaissance,seqn,24,22,-4.974167e+08,24,16,0.916667,0.666667,0.00
...,...,...,...,...,...,...,...,...,...,...,...
23,teaching,renaissance,duet,23,0,-5.885704e+10,23,0,0.000000,0.000000,0.15
24,teaching,renaissance,seqn,24,1,-4.881841e+10,24,0,0.041667,0.000000,0.15
25,teaching,scalabench,duet,21,0,-1.811641e+10,21,0,0.000000,0.000000,0.15
26,teaching,scalabench,seqn,21,2,-1.840440e+10,21,0,0.095238,0.000000,0.15


## MDS using CI-test

In [7]:
px.line(
    df_slowdown,
    x=DF.slowdown,
    y=DF.match_ratio_ci,
    color=RF.type,
    facet_col=DF.env,
    facet_row=RF.suite,
    markers=True,
    height=1000
)

## MDS using u-test

In [11]:
px.line(
    df_slowdown,
    x=DF.slowdown,
    y=DF.match_ratio_utest,
    color=RF.type,
    facet_col=DF.env,
    facet_row=RF.suite,
    markers=True,
    height=1000,
)